# Preprocessing 

In [12]:
import pandas as pd
import os
import re
import numpy as np
import pickle

path = r'C:\Users\chiar\Documents\Università\Text analytics\Data'
#path = r'..\Data'

In [13]:
corpus = pd.read_json(path+'\padriHP.json')
corpus.head() 

,ID,ID_Rif,Titolo_Rif,Rating,Nome_Autore,Data,Numero_Capitoli,Next_ID_List,Position,Racconto_Text_Only,N_Tot_Rec,N_Tot_Rec_Pos,N_Tot_Rec_Neg,Array_Recensioni,This_Rec_Tot,This_Rec_Pos,This_Rec_Cri,This_Rec_Neu
0,3852302,3852302,mi ami?,verde,_SKI_,30/07/2019,2,[3855856],1,1. tre semplici parole\r\n\r\n\r\n-Allora?- Ro...,1,1,0,"[{'rec_autore': 'Sia_', 'rec_giudizio': 'posit...",1,1,0,0
1,2909917,2909917,Rilassati! Hai tutta la morte davanti!,verde,Tonks98,15/11/2014,1,[],1,"""Zi zieda, prego. Allora, quale ezzere zuo nom...",3,3,0,"[{'rec_autore': '_Akari', 'rec_giudizio': 'pos...",3,3,0,0
2,3848998,3848998,Wedding Vows - INTERATTIVA,arancione,laurerichis,13/07/2019,1,[],1,ADELE ROSIER\r\n\r\n \r\nAdele aveva visto mol...,15,15,0,"[{'rec_autore': 'Time_of_the_end', 'rec_giudiz...",15,15,0,0
3,1246472,1246472,"Life is for living, we all know.",giallo,UpSilvietta,04/09/2012,7,"[1247230, 1248201, 1260992, 1265102, 1305903, ...",1,Rose Weasley non era la fotocopia di sua madre...,13,13,0,"[{'rec_autore': 'MYH and ONE', 'rec_giudizio':...",2,2,0,0
4,1390250,1390250,Episodi della Old Generation 1.,verde,mrsreg,17/11/2012,1,[],1,Introduzione.\r\n\r\n\tPersonaggi:\r\n\r\n\tAr...,0,0,0,[],0,0,0,0


In [14]:
corpus.drop(['ID_Rif', 'Next_ID_List', 'Position', 'N_Tot_Rec_Pos', 'N_Tot_Rec_Neg', 'Array_Recensioni',
       'This_Rec_Tot', 'This_Rec_Pos', 'This_Rec_Cri', 'This_Rec_Neu'], axis=1, inplace=True)
corpus.head()

,ID,Titolo_Rif,Rating,Nome_Autore,Data,Numero_Capitoli,Racconto_Text_Only,N_Tot_Rec
0,3852302,mi ami?,verde,_SKI_,30/07/2019,2,1. tre semplici parole\r\n\r\n\r\n-Allora?- Ro...,1
1,2909917,Rilassati! Hai tutta la morte davanti!,verde,Tonks98,15/11/2014,1,"""Zi zieda, prego. Allora, quale ezzere zuo nom...",3
2,3848998,Wedding Vows - INTERATTIVA,arancione,laurerichis,13/07/2019,1,ADELE ROSIER\r\n\r\n \r\nAdele aveva visto mol...,15
3,1246472,"Life is for living, we all know.",giallo,UpSilvietta,04/09/2012,7,Rose Weasley non era la fotocopia di sua madre...,13
4,1390250,Episodi della Old Generation 1.,verde,mrsreg,17/11/2012,1,Introduzione.\r\n\r\n\tPersonaggi:\r\n\r\n\tAr...,0


In [15]:
corpus.rename(columns={"Titolo_Rif": "Title", 
                       "Nome_Autore": "Author",
                       "Data": "Date",
                       "Numero_Capitoli": "Chapter",
                       "Racconto_Text_Only": "Text",
                       "N_Tot_Rec":"N_Rev"}, inplace=True)

In [29]:
df = corpus[corpus['Chapter']==1] 

df.Date = pd.to_datetime(df.Date)
df = df[df.Date.dt.year < 2019]

C:\Users\chiar\AppData\Roaming\Python\Python38\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37693 entries, 1 to 54718
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   ID       37693 non-null  int64         
 1   Title    37690 non-null  object        
 2   Rating   37693 non-null  object        
 3   Author   37693 non-null  object        
 4   Date     37693 non-null  datetime64[ns]
 5   Chapter  37693 non-null  int64         
 6   Text     37693 non-null  object        
 7   N_Rev    37693 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 2.6+ MB


## Cleaning text

In [31]:
df.Text.replace(r'<![\s\S]*?>', ' ', regex=True, inplace=True)
df.Text.replace([r'\\t|\\n|\\r', '\t|\n|\r'], ' ', regex=True, inplace=True) 
df.Text.replace('\u2026', '...', regex=True, inplace=True) 
df.Text.replace(r'_{2,}', ' ', regex=True, inplace=True)
df.Text.replace(r'\.([A-Z][a-z])', r'. \1', regex=True, inplace=True) 
df.Text.replace(r'(\?|\;|\:)([A-Z]|[a-z])', r'\1 \2', regex=True, inplace=True) 
df.Text.replace(r'(\S),(\S)', r'\1, \2', regex=True, inplace=True) 
df.Text.replace(r'(\S)\W,(\S)', r'\1, \2', regex=True, inplace=True) 
df.Text.replace(r'(\S) \.\.\.(\S)', r'\1 ... \2', regex=True, inplace=True) #...v1
df.Text.replace(r'(\S)\.\.\. (\S)', r'\1 ... \2', regex=True, inplace=True) #...v2
df.Text.replace(r'[\{].*?[\}]', r' ', regex=True, inplace=True)
df.Text.replace(r'(((N|n)ome ?(utente)?|Nickname *|(Nick)? ?((A|a)(utore|UTORE)) *:?)(.*))?((T|t)(itolo|ITOLO)(\\i)? *((della)? *(((S|s)toria)|((O|o)riginale)|((T|t)radotto)|((F|f)an *(F|f)iction)|ff|FF|fic|Fic))? *(:|=|-|–))', r' ', regex=True, inplace=True)
df.Text.replace(r'(N|n)uova pagina [0-9]?', r' ', regex=True, inplace=True)
df.Text.replace(r'(C|c)(APITOLO|apitolo) [0-9] ?[:punct:]?', r' ', regex=True, inplace=True)
df.Text.replace(r'@(.*)Section1', r' ', regex=True, inplace=True)
df.Text.replace(r'<(br|\/?(font|a|b|html|body|div|head|title|script))>', r' ', regex=True, inplace=True)
df.Text.replace(r'Leggi le [0-9]? recensioni(.*)Cambia il consenso sulla privacy', r' ', regex=True, inplace=True)
df.Text.replace(r'I personaggi presenti in questa storia sono proprietà di J.K. Rowling', r' ', regex=True, inplace=True)
df.Text.replace(r'(R|r)(ATING|ating)(S|s?) * *(:|=|-|–|è|_) * *(.*?)(((V|v)(erde|ERDE))|((r|R)(osso|OSSO))|R|((A|a)(rancione|RANCIONE))|((G|g)(iallo|IALLO))|((P|p)(er tutti|ER TUTTI))|((P|p)(G|g)) *(13)?|G|M|((N|n)(C|c))(-)?(17)*)', r' ', regex=True, inplace=True)
df.Text.replace(r'(R|r)(ATING|ating) * *(((G|g)(iallo|IALLO))|((R|r)(osso|OSSO))|(A|a)(RANCIONE|rancione)|(V|v)(erde|ERDE)|M|(P|p)er tutti|(PG|pg|Pg)(13)?|G|M|NC(-)?17)', r' ', regex=True, inplace=True)
df.Text.replace(r'((N|n)ome ?(utente)?|Nickname *|(Nick)? ?((A|a)(utore|UTORE))(.*?))(.*?): *[a-zA-Z0-9_]*', r' ', regex=True, inplace=True)
df.Text.replace(r'Nickname (sul forum|su EFP)|Titolo della storia', r' ', regex=True, inplace=True)

In [34]:
df.to_json(path+'\df_final.json', date_format='iso')